# <font color=red>LangChain:  Knowledge Graph Support
- https://docs.langchain.com/docs

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 1 - Simple Knowledge Graph created from text </br>And Used for Inference
</font></span>
This program creates a knowledge graph from text using:  </br>
    &nbsp;&nbsp;&nbsp;&nbsp;
    <font color=lightgreen>graph = index_creator.from_text(text)</font></br>
Note that part of the program is inside an "if False:" which you may want to change to True in order to plot the graph.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.indexes import GraphIndexCreator
from langchain.chains import GraphQAChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name='gpt-4', temperature=0, max_tokens=256)

text = "Apple announced the Vision Pro in 2023."

index_creator = GraphIndexCreator(llm=llm)
graph = index_creator.from_text(text)
print( graph.get_triples() )

chain = GraphQAChain.from_llm(llm, graph=graph, verbose=True)
question = "When did apple announced the Vision Pro?"
response = chain.run(question)
print(response)

if False:   # change to True if you want to draw the graph ########
    import networkx as nx
    import matplotlib.pyplot as plt

    # create graph
    G = nx.DiGraph()
    G.add_edges_from((source, target, {'relation': relation}) for source, relation, target in graph.get_triples())

    # plot the graph
    plt.figure(figsize=(10,8), dpi=100)
    pos = nx.spring_layout(G, k=3, seed=0)

    nx.draw_networkx_nodes(G, pos, node_size=2000)
    nx.draw_networkx_edges(G, pos, edge_color='gray')
    nx.draw_networkx_labels(G, pos, font_size=12)
    edge_labels = nx.get_edge_attributes(G, 'relation')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

    # display the plot
    plt.axis('off')
    plt.show()

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 2 - Larger Knowledge Graph created from triples added individually</br>And Used for Inference
</font></span>
This program creates a knowledge graph from triples added individually using:  </br>
    &nbsp;&nbsp;&nbsp;&nbsp;
    <font color=lightgreen> graph.add_triple(KnowledgeTriple(node1, relation, node2))</font></br>
Note that part of the program is inside an "if False:" which you may want to change to True in order to plot the graph.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.indexes import GraphIndexCreator
from langchain.chains import GraphQAChain
from langchain.prompts import PromptTemplate
from langchain.graphs.networkx_graph import KnowledgeTriple

# Knowledge graph
kg = [
    ('Apple', 'is', 'Company'),
    ('Apple', 'created', 'iMac'),
    ('Apple', 'created', 'iPhone'),
    ('Apple', 'created', 'Apple Watch'),
    ('Apple', 'created', 'Vision Pro'),

    ('Apple', 'developed', 'macOS'),
    ('Apple', 'developed', 'iOS'),
    ('Apple', 'developed', 'watchOS'),

    ('Apple', 'is located in', 'USA'),
    ('Steve Jobs', 'co-founded', 'Apple'),
    ('Steve Wozniak', 'co-founded', 'Apple'),
    ('Tim Cook', 'is the CEO of', 'Apple'),

    ('iOS', 'runs on', 'iPhone'),
    ('macOS', 'runs on', 'iMac'),
    ('watchOS', 'runs on', 'Apple Watch'),

    ('Apple', 'was founded in', '1976'),
    ('Apple', 'owns', 'App Store'),
    ('App Store', 'sells', 'iOS apps'),

    ('iPhone', 'announced in', '2007'),
    ('iMac', 'announced in', '1998'),
    ('Apple Watch', 'announced in', '2014'),
    ('Vision Pro', 'announced in', '2023'),
]

llm = ChatOpenAI(model_name='gpt-4', temperature=0, max_tokens=256)

index_creator = GraphIndexCreator(llm=llm)
graph = index_creator.from_text('')
for (node1, relation, node2) in kg:
    graph.add_triple(KnowledgeTriple(node1, relation, node2))

chain = GraphQAChain.from_llm(llm, graph=graph, verbose=True)
question = "When did apple announce the vision pro?"
print( chain.run(question) )
question = "Who is Tim Cook?"
print( chain.run(question) )

#### useful extras

graph.write_to_gml("graph.gml")
from langchain.indexes.graph import NetworkxEntityGraph
loaded_graph = NetworkxEntityGraph.from_gml("graph.gml")
print(loaded_graph.get_triples())

if False:   # change to True if you want to draw the graph ########
    import networkx as nx
    import matplotlib.pyplot as plt

    G = nx.DiGraph()
    for node1, relation, node2 in kg:
        G.add_edge(node1, node2, label=relation)

    plt.figure(figsize=(10, 8), dpi=100)
    pos = nx.spring_layout(G, k=10, iterations=500, seed=42)
    nx.draw_networkx_nodes(G, pos, node_size=5000)
    nx.draw_networkx_edges(G, pos, edge_color='gray', edgelist=G.edges(), width=2)
    nx.draw_networkx_labels(G, pos, font_size=12)
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=12)
    plt.axis('off')
    plt.show()   # comment this out if not interested

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 3 - One More Small Knowledge Graph Used for Inference
</font></span>
Note that part of the program is inside an "if False:" which you may want to change to True in order to plot the graph.

In [ ]:
from langchain.graphs.networkx_graph import KnowledgeTriple

import networkx as nx
import matplotlib.pyplot as plt

llm = ChatOpenAI(model_name='gpt-4', temperature=0, max_tokens=256)

# Knowledge graph
kg = [
    ("Anne", "is", "cold"),
    ("Anne", "is", "nice"),
    ("Anne", "is", "white"),
    ("Bob", "is", "nice"),
    ("Bob", "is", "white"),
    ("Charlie", "is", "green"),
    ("Charlie", "is", "round"),
    ("Charlie", "is", "white"),
    ("Erin", "is", "big"),
    ("Erin", "is", "cold"),
]

index_creator = GraphIndexCreator(llm=llm)
graph = index_creator.from_text('')
for (node1, relation, node2) in kg:
    graph.add_triple(KnowledgeTriple(node1, relation, node2))

chain = GraphQAChain.from_llm(llm, graph=graph, verbose=True)
question = "Is erin cold?"
print( chain.run(question) )

if False:   # change to True if you want to draw the graph ########
    G = nx.DiGraph()
    for node1, relation, node2 in kg:
        G.add_edge(node1, node2, label=relation)
    plt.figure(figsize=(10, 8), dpi=100)
    pos = nx.spring_layout(G, k=1, iterations=500, seed=0)
    nx.draw_networkx_nodes(G, pos, node_size=5000)
    nx.draw_networkx_edges(G, pos, edge_color='gray', edgelist=G.edges(), width=2)
    nx.draw_networkx_labels(G, pos, font_size=12)
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=12)
    plt.axis('off')
    plt.show()